# Unwrap Context Manager Demonstration

This notebook demonstrates the new `uw.scaled_unwrap()` context manager that provides automatic scaling during unwrap operations.

In [1]:
import sys
sys.path.insert(0, '/Users/lmoresi/+Underworld/underworld-pixi-2/underworld3/src')

import underworld3 as uw
from underworld3.discretisation.enhanced_variables import EnhancedMeshVariable
from underworld3.function.expressions import unwrap

In [2]:
# Setup variables with units
mesh = uw.meshing.StructuredQuadBox(elementRes=(2, 2), minCoords=(0, 0), maxCoords=(1, 1))

temperature = uw.discretisation.MeshVariable("temp", mesh, 1, units="K", units_backend="sympy", varsymbol=r"T")
temperature.set_reference_scaling(1500.0)

velocity = uw.discretisation.MeshVariable("vel", mesh, 2, units="cm/year", units_backend="sympy", varsymbol=r"\mathbf{v}")
velocity.set_reference_scaling(5.0)

print(f"Temperature scale factor: {temperature.scale_factor}")
print(f"Velocity scale factor: {velocity.scale_factor}")

Structured box element resolution 2 2
Temperature scale factor: 0.00100000000000000
Velocity scale factor: 0.100000000000000


## Comparison: Manual vs Automatic Scaling

In [3]:
# Create a complex expression
expr = uw.function.expression("advection", temperature[0] + velocity[0], "advection equation")
print(f"Expression: {expr}")

# Normal unwrap (no scaling)
normal_result = unwrap(expr)
print(f"\nNormal unwrap: {normal_result}")

Expression: UWexpression instance 5, advection

Normal unwrap: { \hspace{ 0.02pt } {T} }(N.x, N.y) + { \hspace{ 0.02pt } {\mathbf{v}} }_{ 0 }(N.x, N.y)


In [4]:
# Method 1: Manual scaling with scaled_symbols()
with uw.scaled_symbols():
    manual_scaled = unwrap(expr)
    print(f"Manual scaling: {manual_scaled}")

Manual scaling: 0.001*{ \hspace{ 0.02pt } {T} }(N.x, N.y) + 0.1*{ \hspace{ 0.02pt } {\mathbf{v}} }_{ 0 }(N.x, N.y)


In [7]:
# Method 2: Automatic scaling with scaled_unwrap()
with uw.apply_scaling():
    auto_scaled = unwrap(expr)
    display(auto_scaled)

0.001*{ \hspace{ 0.02pt } {T} }(N.x, N.y) + 0.1*{ \hspace{ 0.02pt } {\mathbf{v}} }_{ 0 }(N.x, N.y)

In [ ]:
# Verify both methods produce identical results
print(f"Both methods identical: {manual_scaled == auto_scaled}")
print(f"Different from normal: {normal_result != auto_scaled}")

## Multiple Unwrap Operations

In [13]:
# Create multiple expressions
expr1 = uw.function.expression("heat", 2 * temperature, "heat equation")
expr2 = uw.function.expression("flow", velocity[0] + velocity[1], "flow equation")

# Multiple unwrap operations in scaled context
with uw.apply_scaling():
    result1 = unwrap(expr1)
    result2 = unwrap(expr2)
    
    print(f"Heat equation scaled: {result1}")
    print(f"Flow equation scaled: {result2}")

    display(result1)
    display(result2)

Heat equation scaled: Matrix([[0.002*{ \hspace{ 0.02pt } {T} }(N.x, N.y)]])
Flow equation scaled: 0.1*{ \hspace{ 0.02pt } {\mathbf{v}} }_{ 0 }(N.x, N.y) + 0.1*{ \hspace{ 0.02pt } {\mathbf{v}} }_{ 1 }(N.x, N.y)


Matrix([[0.002*{ \hspace{ 0.02pt } {T} }(N.x, N.y)]])

0.1*{ \hspace{ 0.02pt } {\mathbf{v}} }_{ 0 }(N.x, N.y) + 0.1*{ \hspace{ 0.02pt } {\mathbf{v}} }_{ 1 }(N.x, N.y)

## Summary

The `uw.scaled_unwrap()` context manager provides a cleaner API:

- **Automatic**: All unwrap operations within the context are automatically scaled
- **Clean**: No need to remember to wrap each unwrap call
- **Identical Results**: Produces the same scaling as the manual approach
- **Multiple Operations**: Handles multiple unwrap calls within the same context

### Usage Recommendation
- Use `uw.scaled_unwrap()` when you want all unwrap operations to be scaled
- Use `uw.scaled_symbols()` when you need more fine-grained control over when scaling is applied